append-SMB-ATL11
================

Interpolates daily model firn estimates to the coordinates of an ATL11 file

#### Python Dependencies
- [numpy: Scientific Computing Tools For Python](https://numpy.org)  
- [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/netCDF4/index.html)  
- [h5py: Python interface for Hierarchal Data Format 5 (HDF5)](https://www.h5py.org/)  
- [pointCollection: Utilities for organizing and manipulating point data](https://github.com/SmithB/pointCollection)  

This notebook uses Jupyter widgets to set parameters for calculating the firn estimates.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules 

In [ ]:
import os
import re
import SMBcorr
import h5py
import numpy as np
import pointCollection as pc
import ipywidgets as widgets
import matplotlib.pyplot as plt

#### Choose parameters to run
- Input ATL11 file  
- Directory containing SMB models  
- SMB region to run  
- SMB model to run  

In [ ]:
# choose file to read and append
value=''
inputText = widgets.Text(
    value=value,
    description='File:',
    disabled=False
)

# text menu to set model base directory
value=os.getcwd()
directoryText = widgets.Text(
    value=value,
    description='Directory:',
    disabled=False
)

# dropdown menu for setting region
regionDropdown = widgets.Dropdown(
    options=['AA','GL'],
    value='GL',
    description='Region:',
    disabled=False,
)

# dropdown menu for setting SMB model
modelDropdown = widgets.Dropdown(
    options=['MAR','RACMO'],
    value='MAR',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting model version
models = dict(AA={}, GL={})
models['GL']['MAR'] = []
# models['GL']['MAR'].append('MARv3.9-ERA')
# models['GL']['MAR'].append('MARv3.10-ERA')
# models['GL']['MAR'].append('MARv3.11-NCEP')
models['GL']['MAR'].append('MARv3.11-ERA')
models['GL']['MAR'].append('MARv3.11.2-ERA-7.5km')
models['GL']['MAR'].append('MARv3.11.2-ERA-10km')
models['GL']['MAR'].append('MARv3.11.2-ERA-15km')
models['GL']['MAR'].append('MARv3.11.2-ERA-20km')
models['GL']['MAR'].append('MARv3.11.2-NCEP-20km')
models['GL']['RACMO'] = []
# models['GL']['RACMO'].append('RACMO2.3-XGRN11')
# models['GL']['RACMO'].append('RACMO2.3p2-XGRN11')
models['GL']['RACMO'].append('RACMO2.3p2-FGRN055')
versionDropdown = widgets.Dropdown(
    options=models[regionDropdown.value][modelDropdown.value],
    value=models[regionDropdown.value][modelDropdown.value][0],
    description='Version:',
    disabled=False,
)

# function for updating the model version
def set_version(sender):
    versionDropdown.options=models[regionDropdown.value][modelDropdown.value]
    versionDropdown.value=models[regionDropdown.value][modelDropdown.value][0]

# watch widgets for changes
regionDropdown.observe(set_version)
modelDropdown.observe(set_version)

# display widgets for setting parameters
widgets.VBox([inputText,directoryText,regionDropdown,
    modelDropdown,versionDropdown])

#### Function to convert ATL11 time variables

In [ ]:
# PURPOSE: convert time from delta seconds into Julian and year-decimal
def convert_delta_time(delta_time, gps_epoch=1198800018.0):
    # calculate gps time from delta_time
    gps_seconds = gps_epoch + delta_time
    time_leaps = SMBcorr.count_leap_seconds(gps_seconds)
    # calculate julian time
    time_julian = 2444244.5 + (gps_seconds - time_leaps)/86400.0
    # convert to calendar date with convert_julian.py
    Y,M,D,h,m,s = SMBcorr.convert_julian(time_julian,FORMAT='tuple')
    # calculate year-decimal time
    time_decimal = SMBcorr.convert_calendar_decimal(Y,M,DAY=D,HOUR=h,MINUTE=m,SECOND=s)
    # return both the Julian and year-decimal formatted dates
    return dict(julian=time_julian, decimal=time_decimal)

#### Function to set the coordinate projection

In [ ]:
# PURPOSE: set the projection parameters based on the region name
def set_projection(REGION):
    if (REGION == 'AA'):
        projection_flag = 'EPSG:3031'
    elif (REGION == 'GL'):
        projection_flag = 'EPSG:3413'
    return projection_flag

#### Run interpolation program for model and ATL11 file

In [ ]:
# read input file
input_file = os.path.expanduser(inputText.value)
field_dict = {None:('delta_time','h_corr','x','y')}
D11 = pc.data().from_h5(input_file, field_dict=field_dict)
# check if running crossover or along-track ATL11
if (D11.h_corr.ndim == 3):
    nseg,ncycle,ncross = D11.shape
else:
    nseg,ncycle = D11.shape
        
# get projection of input coordinates
EPSG = set_projection(regionDropdown.value)

# extract parameters from widgets
base_dir=os.path.expanduser(directoryText.value)
if (modelDropdown.value == 'MAR'):
    match_object=re.match('(MARv\d+\.\d+(.\d+)?)',versionDropdown.value)
    MAR_VERSION=match_object.group(0)
    MAR_REGION=dict(GL='Greenland',AA='Antarctic')[regionDropdown.value]
    # model subdirectories
    SUBDIRECTORY={}
    SUBDIRECTORY['MARv3.9-ERA']=['ERA_1958-2018_10km','daily_10km']
    SUBDIRECTORY['MARv3.10-ERA']=['ERA_1958-2019-15km','daily_15km']
    SUBDIRECTORY['MARv3.11-NCEP']=['NCEP1_1948-2020_20km','daily_20km']
    SUBDIRECTORY['MARv3.11-ERA']=['ERA_1958-2019-15km','daily_15km']
    SUBDIRECTORY['MARv3.11.2-ERA-7.5km']=['7.5km_ERA5']
    SUBDIRECTORY['MARv3.11.2-ERA-10km']=['10km_ERA5']
    SUBDIRECTORY['MARv3.11.2-ERA-15km']=['15km_ERA5']
    SUBDIRECTORY['MARv3.11.2-ERA-20km']=['20km_ERA5']
    SUBDIRECTORY['MARv3.11.2-NCEP-20km']=['20km_NCEP1']
    MAR_MODEL=SUBDIRECTORY[versionDropdown.value]
    DIRECTORY=os.path.join(base_dir,'MAR',MAR_VERSION,MAR_REGION,*MAR_MODEL)    
    # variable coordinates
    KWARGS = {}
    KWARGS['MARv3.9-ERA'] = dict(XNAME='X10_153',YNAME='Y21_288')
    KWARGS['MARv3.10-ERA'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['MARv3.11-NCEP'] = dict(XNAME='X12_84',YNAME='Y21_155')
    KWARGS['MARv3.11-ERA'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['MARv3.11.2-ERA-7.5km'] = dict(XNAME='X12_203',YNAME='Y20_377')
    KWARGS['MARv3.11.2-ERA-10km'] = dict(XNAME='X10_153',YNAME='Y21_288')
    KWARGS['MARv3.11.2-ERA-15km'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['MARv3.11.2-ERA-20km'] = dict(XNAME='X12_84',YNAME='Y21_155')
    KWARGS['MARv3.11.2-NCEP-20km'] = dict(XNAME='X12_84',YNAME='Y21_155')
    MAR_KWARGS=KWARGS[versionDropdown.value]
elif (modelDropdown.value == 'RACMO'):
    RACMO_VERSION,RACMO_MODEL=versionDropdown.value.split('-')

# check if running crossover or along track
if (D11.h_corr.ndim == 3):
    # allocate for output FIRN height for crossover data 
    FIRN = np.ma.zeros((nseg,ncycle,ncross),fill_value=np.nan)
    FIRN.annual = np.zeros((nseg,ncycle,ncross))
    FIRN.interpolation = np.zeros((nseg,ncycle,ncross),dtype=np.uint8)
    # for each cycle of ICESat-2 ATL11 data
    for c in range(ncycle):
        # check that there are valid crossovers
        cross = [xo for xo in range(ncross) if
            np.any(np.isfinite(D11.delta_time[:,c,xo]))]
        # for each valid crossing
        for xo in cross:
            # find valid crossovers
            i, = np.nonzero(np.isfinite(D11.delta_time[:,c,xo]))
            # convert from delta time to decimal-years
            tdec = convert_delta_time(D11.delta_time[i,c,xo])['decimal']
            if (modelDropdown.value == 'MAR'):
                # read and interpolate daily MAR outputs
                firn_out = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                    MAR_VERSION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='ZN6', SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
            elif (modelDropdown.value == 'RACMO'):
                # read and interpolate daily RACMO outputs
                firn_out = SMBcorr.interpolate_racmo_daily(base_dir, EPSG,
                    RACMO_MODEL, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='hgtsrf', SIGMA=1.5, FILL_VALUE=np.nan)
            # set attributes to output for iteration
            FIRN[i,c,xo] = np.copy(firn_out.data)
            FIRN.annual[i,c,xo] = np.copy(firn_out.annual)
            FIRN.interpolation[i,c,xo] = np.copy(firn_out.interpolation)
else:        
    # for each cycle of ICESat-2 ATL11 data
    FIRN = np.ma.zeros((nseg,ncycle),fill_value=np.nan)
    FIRN.annual = np.zeros((nseg,ncycle))
    FIRN.interpolation = np.zeros((nseg,ncycle),dtype=np.uint8)    
    # check that there are valid elevations
    cycle = [c for c in range(ncycle) if
        np.any(np.isfinite(D11.delta_time[:,c]))]
    # for each valid cycle of ICESat-2 ATL11 data
    for c in cycle:
        # find valid elevations
        i, = np.nonzero(np.isfinite(D11.delta_time[:,c]))
        # convert from delta time to decimal-years
        tdec = convert_delta_time(D11.delta_time[i,c])['decimal']
        if (modelDropdown.value == 'MAR'):
            # read and interpolate daily MAR outputs
            firn_out = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                MAR_VERSION, tdec, D11.x[i,c], D11.y[i,c], VARIABLE='ZN6',
                SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
        elif (modelDropdown.value == 'RACMO'):
            # read and interpolate daily RACMO outputs
            firn_out = SMBcorr.interpolate_racmo_daily(base_dir, EPSG,
                RACMO_MODEL, tdec, D11.x[i,c], D11.y[i,c], VARIABLE='hgtsrf',
                SIGMA=1.5, FILL_VALUE=np.nan)
        # set attributes to output for iteration
        FIRN[i,c] = np.copy(firn_out.data)
        FIRN.annual[i,c] = np.copy(firn_out.annual)
        FIRN.interpolation[i,c] = np.copy(firn_out.interpolation)
            
# replace mask values
FIRN.mask = (FIRN.data == FIRN.fill_value)

#### Create plot showing elevation change and firn height change

In [ ]:
if (D11.h_corr.ndim == 2):
    fig,(ax1,ax2) = plt.subplots(ncols=2)
    c1,c2 = (0,1)
    i, = np.nonzero(np.isfinite(D11.h_corr[:,c1]) & np.isfinite(D11.h_corr[:,c2]) & 
        (~FIRN.mask[:,c1]) & (~FIRN.mask[:,c2]))
    # height change
    ax1.scatter(D11.x[i,c1],D11.y[i,c1],
        c=D11.h_corr[i,c2]-D11.h_corr[i,c1],
        vmin=-5,vmax=0.5,s=0.5)
    ax1.set_title = 'ATL11'
    # firn height change
    ax2.scatter(D11.x[i,c1],D11.y[i,c1],
        c=FIRN[i,c2]-FIRN[i,c1],
        vmin=-5,vmax=0.5,s=0.5)
    ax2.set_title = versionDropdown.value
    plt.show()

#### Append data to input ATL11 HDF5 file

In [ ]:
# append input HDF5 file with new firn model outputs
fileID = h5py.File(os.path.expanduser(inputText.value),'a')
fileID.create_group(versionDropdown.value)
h5 = {}
val = '{0}/{1}'.format(versionDropdown.value,'zsurf')
h5['zsurf'] = fileID.create_dataset(val, FIRN.shape, data=FIRN,
    dtype=FIRN.dtype, compression='gzip', fillvalue=FIRN.fill_value)
h5['zsurf'].attrs['units'] = "m"
h5['zsurf'].attrs['long_name'] = "Snow Height Change"
h5['zsurf'].attrs['coordinates'] = "../delta_time ../latitude ../longitude"
h5['zsurf'].attrs['model'] = versionDropdown.value
val = '{0}/{1}'.format(versionDropdown.value,'zannual')
h5['zannual'] = fileID.create_dataset(val, FIRN.shape, data=FIRN.annual,
    dtype=FIRN.dtype, compression='gzip', fillvalue=FIRN.fill_value)
h5['zannual'].attrs['units'] = "m"
h5['zannual'].attrs['long_name'] = "Annual Snow Height Change"
h5['zannual'].attrs['coordinates'] = "../delta_time ../latitude ../longitude"
h5['zannual'].attrs['model'] = versionDropdown.value
fileID.close()